In [1]:
import os
import cv2
import numpy as np
from sklearn import preprocessing

def printMatrix(matrix):
    rows,col = matrix.shape
    print("printMatrix - Row: ", str(rows), "Column: ", str(col))
    lines = []
    for row in matrix:
        lines.append(' '.join(str(x) for x in row))
    print('\n'.join(lines))
    
## MATRIX X e Y com tamanhos diferentes!
generated_filepath = os.getcwd() + '//Generated//'
n_signs = 29
#row_count = 0

y_matrix = np.empty([0, n_signs])
x_matrix = []
for sign_count in range(0,n_signs):
        inputdir =  generated_filepath + str(sign_count) + '//'
        y_sign = np.zeros((1,n_signs), dtype=np.int)
        y_sign[0][sign_count] = 1
        for image in os.listdir(inputdir):
            imageFilepath = inputdir + image
            img_matrix = cv2.imread(imageFilepath,0)
            img_array = np.ravel(img_matrix)
            y_matrix = np.concatenate((y_matrix, y_sign), axis=0)
            x_matrix.append(img_array)
            #row_count += 1
x_matrix = np.array(x_matrix)

min_max_scaler = preprocessing.MinMaxScaler()
x_matrix_scale = min_max_scaler.fit_transform(x_matrix)
#x_matrix_scale = x_matrix
#x_matrix_scale *= 255.0/x_matrix.max()
print("Dataset created in 'x_matrix' (normal) and 'x_matrix_scale' (normalized) as input and 'y_matrix' as output")
%timeit

/root/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


Dataset created in 'x_matrix' (normal) and 'x_matrix_scale' (normalized) as input and 'y_matrix' as output


In [15]:
from neon.callbacks.callbacks import Callbacks
#from neon.data import MNIST
from neon.initializers import Gaussian
from neon.layers import GeneralizedCost, Affine
from neon.models import Model
from neon.optimizers import GradientDescentMomentum
from neon.transforms import Rectlin, Logistic, CrossEntropyMulti, CrossEntropyBinary, Misclassification
from neon.util.argparser import NeonArgparser
from neon import logger as neon_logger
from neon.data.dataiterator import ArrayIterator
from neon.backends import gen_backend
from sklearn.model_selection import train_test_split

# parse the command line arguments
#parser = NeonArgparser(__doc__)

#args = parser.parse_args()

# load up the mnist data set

gen_backend(backend='cpu', batch_size=128)
X_train, X_test, Y_train, Y_test = train_test_split(x_matrix_scale, y_matrix, test_size=0.33)

train_set = ArrayIterator(X=X_train, y=Y_train, nclass=29, lshape=(1,56,56))
valid_set = ArrayIterator(X=X_test, y=Y_test, nclass=29, lshape=(1,56,56))

#dataset = {'train': (X_train, Y_train),'test': (X_test, Y_test)}

#train_set = ArrayIterator(X=X_train, y=Y_train, nclass=30, make_onehot=True)
#valid_set = ArrayIterator(X=X_test, y=Y_test, nclass=30, make_onehot=True)


# setup weight initialization function
init_norm = Gaussian(loc=0.0, scale=0.01)

# setup model layers
n_input = np.shape(x_matrix_scale)[1]
y_output = np.shape(y_matrix)[1]
n_layer_1 = 300
n_layer_2 = 300
n_epochs = 100
layers = [#Affine(nout=n_input, init=init_norm, activation=Rectlin()),
          Affine(nout=n_layer_1, init=init_norm, activation=Rectlin()),
          Affine(nout=n_layer_2, init=init_norm, activation=Rectlin()),
          Affine(nout=y_output, init=init_norm, activation=Logistic(shortcut=True))]

# setup cost function as CrossEntropy
cost = GeneralizedCost(costfunc=CrossEntropyBinary())

# setup optimizer
optimizer = GradientDescentMomentum(
    0.1, momentum_coef=0.9)

# initialize model object
mlp = Model(layers=layers)

# configure callbacks
callbacks = Callbacks(mlp, eval_set=valid_set)
#callbacks = Callbacks(mlp, eval_set=dataset['test'])

# run fit
mlp.fit(train_set, optimizer=optimizer,num_epochs=n_epochs, cost=cost, callbacks=callbacks)
#mlp.fit(dataset['test'], optimizer=optimizer,num_epochs=n_epochs, cost=cost, callbacks=callbacks)
error_rate = mlp.eval(valid_set, metric=Misclassification())
#error_rate = mlp.eval(dataset['test']), metric=Misclassification())
neon_logger.display('Misclassification error = %.1f%%' % (error_rate * 100))


Epoch 0   [Train |████████████████████|   77/77   batches, 16.15 cost, 2.03s]
Epoch 1   [Train |████████████████████|   77/77   batches, 20.10 cost, 1.98s]
Epoch 2   [Train |████████████████████|   77/77   batches, 20.10 cost, 1.93s]
Epoch 3   [Train |████████████████████|   77/77   batches, 20.10 cost, 1.94s]
Epoch 4   [Train |████████████████████|   77/77   batches, 20.10 cost, 1.97s]
Epoch 5   [Train |████████████████████|   77/77   batches, 20.10 cost, 1.98s]
Epoch 6   [Train |████████████████████|   77/77   batches, 20.10 cost, 1.95s]
Epoch 7   [Train |████████████████████|   77/77   batches, 20.10 cost, 1.97s]
Epoch 8   [Train |████████████████████|   77/77   batches, 20.10 cost, 1.95s]
Epoch 9   [Train |████████████████████|   76/76   batches, 20.10 cost, 2.10s]
Epoch 10  [Train |████████████████████|   77/77   batches, 20.10 cost, 2.69s]
Epoch 11  [Train |█████████           |   36/77   batches, 20.10 cost, 1.42s]

KeyboardInterrupt: 

In [18]:
import urllib
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline 

# download image
url = "https://www.seton.com.br/media/catalog/product/cache/1/image/1200x1200/9df78eab33525d08d6e5fb8d27136e95/c/0/c0247-ba.jpg"
urllib.urlretrieve(url, filename="data/digit.jpg")

# scale to 28x28 pixels
img = cv2.imread("data/digit.jpg",0)
img.thumbnail((56, 56))

digit = np.asarray(img, dtype=np.float32)[:, :, 0]

# reshape to a single feature vector
digit = digit.reshape(1,3136)

# store digit into a GPU tensor
x_new = be.zeros((28*28, 128), dtype=np.float32)
x_new[:, 0] = digit


AttributeError: Must provide number of classes when creating onehot labels

In [22]:
# examine the output of the model for this image
print "Model final layer was: {}".format(outputs)
print "The most probable guess is digit: {}".format(np.argmax(outputs))
plt.figure(2)
plt.imshow(img)


(14690, 3136)
(14690, 29)


In [5]:
train_set = ArrayIterator(X=X_train, y=Y_train, nclass=29)#, lshape=(1,56,56))
valid_set = ArrayIterator(X=X_test, y=Y_test, nclass=29)#, lshape=(1,56,56))
print(train_set.shape)
print(valid_set.shape)
print(X_train.shape)
print(train_set)

3136
3136
(9842, 3136)


In [54]:
datsets = {'train': (X_train, Y_train),'test': (X_test, Y_test)}
from neon.data import MNIST
mnist = MNIST(path='data/')

train_set = mnist.train_iter
valid_set = mnist.valid_iter

print(type(train_set))
print((train_set).shape)
print(type(datsets['train']))

<class 'neon.data.dataiterator.ArrayIterator'>
(1, 28, 28)
<class 'tuple'>


In [ ]:
 import some useful packages
from neon.data import NervanaDataIterator
import numpy as np
import cPickle
import os

class TSR(NervanaDataIterator):

    def __init__(self, X, Y, lshape):

        # Load the numpy data into some variables. We divide the image by 255 to normalize the values
        # between 0 and 1.
        self.X = X / 255.
        self.Y = Y
        self.shape = lshape  # shape of the input data (e.g. for images, (C, H, W))

        # 1. assign some required and useful attributes
        self.start = 0  # start at zero
        self.ndata = X.shape(1) # number of images in X (hint: use X.shape)
        self.nfeatures = X.shape(2)  # number of features in X (hint: use X.shape)

        # number of minibatches per epoch
        # to calculate this, use the batchsize, which is stored in self.be.bsz
        self.nbatches = self.be.bsz
        
        
        # 2. allocate memory on the GPU for a minibatch's worth of data.
        # (e.g. use `self.be` to access the backend.). See the backend documentation.
        # to get the minibatch size, use self.be.bsz
        # hint: X should have shape (# features, mini-batch size)
        # hint: use some of the attributes previously defined above
        self.dev_X = ...
        self.dev_Y = ...


    def reset(self):
        self.start = 0

    def __iter__(self):
        # 3. loop through minibatches in the dataset
        for index in range(self.start, self.ndata, self.be.bsz):
            # 3a. grab the right slice from the numpy arrays
            inputs = ...
            targets = ...
            
            # The arrays X and Y data are in shape (batch_size, num_features),
            # but the iterator needs to return data with shape (num_features, batch_size).
            # here we transpose the data, and then store it as a contiguous array. 
            # numpy arrays need to be contiguous before being loaded onto the GPU.
            inputs = np.ascontiguousarray(inputs.T)
            targets = np.ascontiguousarray(targets.T)
                        
            # here we test your implementation
            # your slice has to have the same shape as the GPU tensors you allocated
            assert inputs.shape == self.dev_X.shape, \
                   "inputs has shape {}, but dev_X is {}".format(inputs.shape, self.dev_X.shape)
            assert targets.shape == self.dev_Y.shape, \
                   "targets has shape {}, but dev_Y is {}".format(targets.shape, self.dev_Y.shape)
            
            # 3b. transfer from numpy arrays to device
            # - use the GPU memory buffers allocated previously,
            #    and call the myTensorBuffer.set() function. 
            self.dev_X ...
            self.dev_Y ...
            
            # 3c. yield a tuple of the device tensors.
            # the first should of shape (num_features, batch_size)
            # the second should of shape (4, batch_size)
            yield (..., ...)

In [14]:
train_set = ArrayIterator(X=X_train, y=Y_train, nclass=29, lshape=(1,56,56))
print(train_set.shape)
print(type(train_set))
test = train_set.__iter__()

n_input = np.shape(x_matrix_scale)[1]
y_output = np.shape(y_matrix)[1]
print(n_input)
print(y_output)

(1, 56, 56)
<class 'neon.data.dataiterator.ArrayIterator'>
3136
29
